In [13]:
%load_ext ngql

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql


In [16]:
import os
import logging
import sys

import openai
import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown, display
from llama_index.core import load_index_from_storage
from llama_index.core import (
    Document,
    KnowledgeGraphIndex,
    StorageContext,
)
from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.llms.openai import OpenAI
from llama_index.graph_stores.nebula import NebulaGraphStore
from llama_index.legacy.query_engine.knowledge_graph_query_engine import DEFAULT_NEO4J_NL2CYPHER_PROMPT


load_dotenv()

logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
)

openai.api_key = os.environ["OPENAI_API_KEY"]


In [17]:
space_name = "policies_aa"
edge_types, rel_prop_names = ["relationship"], [
    "name"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

storage_context = StorageContext.from_defaults(
    persist_dir="./data/index/policies_aa",
    graph_store=graph_store
)

kg_index = load_index_from_storage(
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ./data/index/policies_aa/docstore.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg/data/index/policies_aa/docstore.json
DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ./data/index/policies_aa/index_store.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg/data/index/policies_aa/index_store.json
DEBUG:llama_index.core.vector_stores.simple:Loading llama_index.core.vector_stores.simple from ./data/index/policies_aa/image__vector_store.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg/data/index/policies_aa/image__vector_store.json
DEBUG:llama_index.core.vector_stores.simple:Loading llama_index.core.vector_stores.simple from ./data/index/policies_aa/default__vector_store.json.
DEBUG:fsspec.local:

In [78]:

llm = OpenAI(temperature=0, model="gpt-3.5-turbo") #  model="gpt-3.5-turbo")

nl2kg_query_engine = KnowledgeGraphQueryEngine(
    graph_query_synthesis_prompt=DEFAULT_NEO4J_NL2CYPHER_PROMPT,
    llm=llm,
    refresh_schema=True,
    storage_context=storage_context,
    verbose=True,
)

DEBUG:llama_index.graph_stores.nebula.base:get_schema()
schema: Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]
Edge properties: [{'edge': 'relationship', 'properties': [('name', 'string')]}]
Relationships: ['(:entity)-[:relationship]->(:entity)']



In [25]:
question = """
Tell me about Cuba?
"""

response_nl2kg = nl2kg_query_engine.query(question)

# Cypher:

print("The Cypher Query is:")

query_string = nl2kg_query_engine.generate_query(question)

display(
    Markdown(
        f"""
```cypher
{query_string}
```
"""
    )
)

%ngql policies_aa {query_string}

# Answer:

print("The Answer is:")

display(Markdown(f"<b>{response_nl2kg}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\nNode properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]\nEdge properties: [{'edge': 'relationship', 'properties': [('name', 'string')]}]\nRelationships: ['(:entity)-[:relationship]->(:entity)']\n\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement. \nDo not include any text except the generated Cypher statement.\n\nThe question is:\n\nTell me about Cuba?\n\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.0}}
DEBUG:openai._base_client:Sending 


```cypher
MATCH (:entity {name: 'Cuba'})-[:relationship]->(related_entity)
RETURN related_entity;
```


INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
[ERROR]:
 Query Failed:
 SyntaxError: syntax error near `policies_aa'
The Answer is:


<b>Cuba is related to entities such as Bag fees, Bag limitations, Bag 1, Bag 2, Bag 3, Bag 4+, Trips, and Checked bag policy.</b>

In [26]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

In [27]:
response_graph_rag = kg_index_query_engine.query("Tell me about baggage policies on flights to Cuba")

display(Markdown(f"<b>{response_graph_rag}</b>"))

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/pez/.local/share/virtualenvs/nlp_airline_project-fCxuAmfF/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nTell me about baggage policies on flights to Cuba\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.

<b>Baggage policies on flights to Cuba include the allowance for each passenger to check up to 2 bags with a maximum weight of 70 lbs per bag. Additionally, passengers are allowed 1 carry-on bag and 1 personal item. There are specific fees associated with checked bags when traveling to or from Cuba, with different fees for the 2nd bag depending on the travel dates and destination. It is important to note that all published bag fees are non-refundable and apply per person, each way.</b>

In [29]:
response = kg_index.as_retriever().retrieve("Tell me about baggage policies on flights to Cuba")
display(Markdown(f"<b>{response}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nTell me about baggage policies on flights to Cuba\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.Sy

<b>[NodeWithScore(node=TextNode(id_='198e667a-b41e-49c5-9268-34cbdcdc98cd', embedding=None, metadata={'airline': 'American Airlines', 'policy': 'Checked Bag Policy', 'topic': 'Cuba'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='61cb35e0-39fc-4f7e-b66c-495c663289ec', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'American Airlines', 'policy': 'Checked Bag Policy', 'topic': 'Cuba'}, hash='5e0b2ceb80615ab0d03983f90938d4ed7017dbceae07cf270dad2513b16ef075'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='4181e499-6b9e-43f2-b922-f2a72ab03aea', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'American Airlines', 'policy': 'Checked Bag Policy', 'topic': 'Cuba'}, hash='dc435636892e64a0aa075054f299cfa099de1e8ae8c56c8620531ba56d92d945'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='0bdf12a9-56d6-4369-8655-4ce17160611b', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='0a614895fe6820bf22c55253f3a0c0b0067f4adecb4a584960cb62c8ef1be45f')}, text='For tickets issued on / after September 20, 2023 a $30 1st bag fee applies to Basic Economy and there is no 1st bag fee for Main Cabin. For tickets issued on / after February 20, 2024 a $35 1st bag fee applies to Basic Economy and there is no 1st bag fee for Main Cabin.  \n**For travel to Cuba, a $200 2nd bag fee applies seasonally for travel from November 16 – January 9. A $150 2nd bag fee applies otherwise. Excludes Main Plus for tickets issued on / after September 20, 2023.  \n^Main Plus includes 1 extra free checked bag in addition to the Main Cabin allowance (max of 2).  \nAll published bag fees apply at each check-in location and are base rates according to travel dates and destination; applicable taxes are not shown.  \nAll bag fees are non-refundable and apply per person, each way, even if you buy or get an upgrade that includes free checked bags.', start_char_idx=3032, end_char_idx=3895, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='508a2aee-b4b9-4273-898d-c3d101daca5c', embedding=None, metadata={'airline': 'American Airlines', 'policy': 'Checked Bag Policy', 'topic': 'Cuba'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='61cb35e0-39fc-4f7e-b66c-495c663289ec', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'American Airlines', 'policy': 'Checked Bag Policy', 'topic': 'Cuba'}, hash='5e0b2ceb80615ab0d03983f90938d4ed7017dbceae07cf270dad2513b16ef075'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='a4b6d678-7fcf-4531-9b2d-7501c1fb6ba4', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='46c7c44a55f1e0a62aaaca3947a263087fa45394009403c422e6a9b2ab13327c')}, text='Checked Bag Policy\nCuba\nThrough checked bags\nEach passenger traveling to Cuba may only check up to 2 bags* with a maximum weight of 70 lbs / 32 kgs per bag, plus 1 carry-on bag and 1 personal item.  \nBag limitations Carry-on bags  \n*Seasonal exceptions apply to Havana, Cuba effective for travel on / after March 14, 2023\n\nEach passenger traveling to Cuba may only check up to 2 bags* with a maximum weight of 70 lbs / 32 kgs per bag, plus 1 carry-on bag and 1 personal item.', start_char_idx=0, end_char_idx=475, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='4181e499-6b9e-43f2-b922-f2a72ab03aea', embedding=None, metadata={'airline': 'American Airlines', 'policy': 'Checked Bag Policy', 'topic': 'Cuba'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='61cb35e0-39fc-4f7e-b66c-495c663289ec', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'airline': 'American Airlines', 'policy': 'Checked Bag Policy', 'topic': 'Cuba'}, hash='5e0b2ceb80615ab0d03983f90938d4ed7017dbceae07cf270dad2513b16ef075'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='6e6a9958-70ba-4bfb-bd5c-d497d3649e57', node_type=<ObjectType.TEXT: '1'>, metadata={'airline': 'American Airlines', 'policy': 'Checked Bag Policy', 'topic': 'Cuba'}, hash='9a4371cad5c3271d0e67363c20f812cded22dd9bc6e8cd92fc4c6aa2cc0c4656'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='198e667a-b41e-49c5-9268-34cbdcdc98cd', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='004ede72f7c22a9f8c3091d320c64a72f18bddeaa1b031f3c1d26fce68b2755a')}, text='Trips originating in Cuba Bag 1 Bag 2 Bag 3 Bag 4+ From Cuba $0 $0 $200 $200 To Cuba $0 $150 / $200** $200 – Not available Trips originating outside Cuba Bag 1 Bag 2 Bag 3 Bag 4+ To Cuba $0 / $35*^ $150 / $200**^ $200 – Not available From Cuba $0 $0 $200 $200  \nAll published bag fees apply at each check-in location and are base rates according to travel dates and destination; applicable taxes are not shown.  \nAll bag fees are non-refundable and apply per person, each way, even if you buy or get an upgrade that includes free checked bags. If you believe you were incorrectly charged for bag fees, contact an American representative for help or file a refund claim within 45 days.  \nOversize and overweight bags Bag and optional fees Refunds  \n*For travel to Cuba, a $30 1st bag fee applies to Basic Economy and Main Cabin for tickets issued on / before September 19, 2023.', start_char_idx=694, end_char_idx=1571, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0), NodeWithScore(node=TextNode(id_='f9712ccb-5cb5-4da8-9629-695423c81f74', embedding=None, metadata={'kg_rel_texts': ['Cuba{name: Cuba} -[relationship:{name: Is topic of}]-> Checked bag policy{name: Checked bag policy} -[relationship:{name: Includes}]-> Special items and sports equipment{name: Special items and sports equipment}', 'Cuba{name: Cuba} -[relationship:{name: Is topic of}]-> Checked bag policy{name: Checked bag policy} -[relationship:{name: Includes}]-> Paying for bags online{name: Paying for bags online}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> 1 personal item and 1 carry-on{name: 1 personal item and 1 carry-on}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Flying on a partner airline{name: Flying on a partner airline}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> How many bags can you take{name: How many bags can you take}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Find your trip on aa.com{name: Find your trip on aa.com}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Applies to}]-> Travel to south america{name: Travel to south america}', 'Cuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply at}]-> Each check-in location{name: Each check-in location}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Checked bag allowances{name: Checked bag allowances}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Customer service faqs{name: Customer service faqs}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> What can you fly with{name: What can you fly with}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> How many bags can you take{name: How many bags can you take}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Checked bag allowances{name: Checked bag allowances}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is}]-> Through checked bags{name: Through checked bags}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Applies to}]-> Transpacific travel{name: Transpacific travel}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> British airways{name: British airways}', 'Cuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply per person}]-> Each way{name: Each way}', 'Cuba{name: Cuba} -[relationship:{name: Originating in}]-> Trips{name: Trips} <-[relationship:{name: Originating outside}]- Cuba{name: Cuba}', 'Cuba{name: Cuba} -[relationship:{name: Originating outside}]-> Bag 1{name: Bag 1} <-[relationship:{name: Originating in}]- Cuba{name: Cuba}', 'Cuba{name: Cuba} -[relationship:{name: Originating outside}]-> Trips{name: Trips} <-[relationship:{name: Originating in}]- Cuba{name: Cuba}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Delayed bags{name: Delayed bags}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Damaged bags{name: Damaged bags}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> You may also like{name: You may also like}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Receipts{name: Receipts}', 'Cuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply per}]-> Person{name: Person}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Finnair{name: Finnair}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> Delayed bags{name: Delayed bags}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Iberia{name: Iberia}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Qantas{name: Qantas}', 'Cuba{name: Cuba} -[relationship:{name: Originating in}]-> Bag 4+{name: Bag 4+}'], 'kg_rel_map': {'Cuba{name: Cuba}': ['Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Iberia{name: Iberia}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Finnair{name: Finnair}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Customer service faqs{name: Customer service faqs}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> British airways{name: British airways}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Applies to}]-> Travel to south america{name: Travel to south america}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> What can you fly with{name: What can you fly with}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Receipts{name: Receipts}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> How many bags can you take{name: How many bags can you take}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Flying on a partner airline{name: Flying on a partner airline}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Delayed bags{name: Delayed bags}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Checked bag allowances{name: Checked bag allowances}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is}]-> Through checked bags{name: Through checked bags}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Applies to}]-> Transpacific travel{name: Transpacific travel}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> How many bags can you take{name: How many bags can you take}', 'Cuba{name: Cuba} -[relationship:{name: Originating in}]-> Bag 4+{name: Bag 4+}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> Delayed bags{name: Delayed bags}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Find your trip on aa.com{name: Find your trip on aa.com}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> 1 personal item and 1 carry-on{name: 1 personal item and 1 carry-on}', 'Cuba{name: Cuba} -[relationship:{name: Originating outside}]-> Trips{name: Trips} <-[relationship:{name: Originating in}]- Cuba{name: Cuba}', 'Cuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply at}]-> Each check-in location{name: Each check-in location}', 'Cuba{name: Cuba} -[relationship:{name: Originating outside}]-> Bag 1{name: Bag 1} <-[relationship:{name: Originating in}]- Cuba{name: Cuba}', 'Cuba{name: Cuba} -[relationship:{name: Is topic of}]-> Checked bag policy{name: Checked bag policy} -[relationship:{name: Includes}]-> Special items and sports equipment{name: Special items and sports equipment}', 'Cuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply per}]-> Person{name: Person}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Qantas{name: Qantas}', 'Cuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply per person}]-> Each way{name: Each way}', 'Cuba{name: Cuba} -[relationship:{name: Is topic of}]-> Checked bag policy{name: Checked bag policy} -[relationship:{name: Includes}]-> Paying for bags online{name: Paying for bags online}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Checked bag allowances{name: Checked bag allowances}', 'Cuba{name: Cuba} -[relationship:{name: Originating in}]-> Trips{name: Trips} <-[relationship:{name: Originating outside}]- Cuba{name: Cuba}', 'Cuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> You may also like{name: You may also like}', 'Cuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Damaged bags{name: Damaged bags}']}, 'kg_schema': {'schema': "Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]\nEdge properties: [{'edge': 'relationship', 'properties': [('name', 'string')]}]\nRelationships: ['(:entity)-[:relationship]->(:entity)']\n"}}, excluded_embed_metadata_keys=['kg_rel_map', 'kg_rel_texts'], excluded_llm_metadata_keys=['kg_rel_map', 'kg_rel_texts'], relationships={}, text='The following are knowledge sequence in max depth 2 in the form of directed graph like:\n`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`\nCuba{name: Cuba} -[relationship:{name: Is topic of}]-> Checked bag policy{name: Checked bag policy} -[relationship:{name: Includes}]-> Special items and sports equipment{name: Special items and sports equipment}\nCuba{name: Cuba} -[relationship:{name: Is topic of}]-> Checked bag policy{name: Checked bag policy} -[relationship:{name: Includes}]-> Paying for bags online{name: Paying for bags online}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> 1 personal item and 1 carry-on{name: 1 personal item and 1 carry-on}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Flying on a partner airline{name: Flying on a partner airline}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> How many bags can you take{name: How many bags can you take}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Find your trip on aa.com{name: Find your trip on aa.com}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Applies to}]-> Travel to south america{name: Travel to south america}\nCuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply at}]-> Each check-in location{name: Each check-in location}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Checked bag allowances{name: Checked bag allowances}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Customer service faqs{name: Customer service faqs}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> What can you fly with{name: What can you fly with}\nCuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> How many bags can you take{name: How many bags can you take}\nCuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Checked bag allowances{name: Checked bag allowances}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is}]-> Through checked bags{name: Through checked bags}\nCuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Applies to}]-> Transpacific travel{name: Transpacific travel}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> British airways{name: British airways}\nCuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply per person}]-> Each way{name: Each way}\nCuba{name: Cuba} -[relationship:{name: Originating in}]-> Trips{name: Trips} <-[relationship:{name: Originating outside}]- Cuba{name: Cuba}\nCuba{name: Cuba} -[relationship:{name: Originating outside}]-> Bag 1{name: Bag 1} <-[relationship:{name: Originating in}]- Cuba{name: Cuba}\nCuba{name: Cuba} -[relationship:{name: Originating outside}]-> Trips{name: Trips} <-[relationship:{name: Originating in}]- Cuba{name: Cuba}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Delayed bags{name: Delayed bags}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Damaged bags{name: Damaged bags}\nCuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> You may also like{name: You may also like}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Is about}]-> Receipts{name: Receipts}\nCuba{name: Cuba} -[relationship:{name: Has}]-> Bag fees{name: Bag fees} -[relationship:{name: Apply per}]-> Person{name: Person}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Finnair{name: Finnair}\nCuba{name: Cuba} <-[relationship:{name: Is}]- Topic{name: Topic} -[relationship:{name: Is}]-> Delayed bags{name: Delayed bags}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Iberia{name: Iberia}\nCuba{name: Cuba} <-[relationship:{name: Is about}]- Topic{name: Topic} -[relationship:{name: Includes}]-> Qantas{name: Qantas}\nCuba{name: Cuba} -[relationship:{name: Originating in}]-> Bag 4+{name: Bag 4+}', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1000.0)]</b>

In [46]:
from pprint import pprint

for node in response:
    print("NODE:")
    pprint(node.dict())
    if hasattr(node, "relationships"):
        for key, rel in node.relationships.items():
            print("RELATION: ", key)
            pprint(rel.dict())


NODE:
{'class_name': 'NodeWithScore',
 'node': {'class_name': 'TextNode',
          'embedding': None,
          'end_char_idx': 3895,
          'excluded_embed_metadata_keys': [],
          'excluded_llm_metadata_keys': [],
          'id_': '198e667a-b41e-49c5-9268-34cbdcdc98cd',
          'metadata': {'airline': 'American Airlines',
                       'policy': 'Checked Bag Policy',
                       'topic': 'Cuba'},
          'metadata_seperator': '\n',
          'metadata_template': '{key}: {value}',
          'relationships': {<NodeRelationship.SOURCE: '1'>: {'class_name': 'RelatedNodeInfo',
                                                             'hash': '5e0b2ceb80615ab0d03983f90938d4ed7017dbceae07cf270dad2513b16ef075',
                                                             'metadata': {'airline': 'American '
                                                                                     'Airlines',
                                                       

-----

In [49]:
question = "What is the baggage fee for economy class on a flight to Cuba?"
response_graph_rag = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>\n\n<b>A: {response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the baggage fee for economy class on a flight to Cuba?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._bac

<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>

<b>A: The baggage fee for economy class on a flight to Cuba is $200 for the second bag during the seasonal period from November 16 to January 9. Outside of this seasonal period, the fee is $150 for the second bag.</b>

In [51]:
question = "What policies does American Airlines have?"
response_graph_rag = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat policies does American Airlines have?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStrea

<b>Q: What policies does American Airlines have?<b>

<b>A: American Airlines has a policy that does not allow passengers to transport marijuana on their flights. This includes both recreational and medical marijuana, as it is considered illegal under federal law. Passengers traveling with marijuana on American Airlines flights do so at their own risk.</b>

In [52]:
question = "What is not allowed on American Airlines?"
response_graph_rag = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>\n\n<b>A: {response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is not allowed on American Airlines?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream

<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>

<b>A: Transporting marijuana, including recreational and medical marijuana, is not allowed on American Airlines.</b>

In [53]:
question = "Are batteries allowed on flights?"
response_graph_rag = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>\n\n<b>A: {response_graph_rag}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nAre batteries allowed on flights?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object 

<b>Q:What is the baggage fee for economy class on a flight to Cuba?<b>

<b>A: Yes, batteries are generally allowed on flights, but there are restrictions on the types and quantities of batteries that can be brought on board. It is recommended to check with the specific airline for their policies on carrying batteries.</b>

In [55]:
question = "Are batteries allowed on flights?"
response = kg_index.as_retriever().retrieve(question)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nAre batteries allowed on flights?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object 

In [56]:
for node in response:
    print("NODE:")
    pprint(node.dict())
    if hasattr(node, "relationships"):
        for key, rel in node.relationships.items():
            print("RELATION: ", key)
            pprint(rel.dict())


NODE:
{'class_name': 'NodeWithScore',
 'node': {'class_name': 'TextNode',
          'embedding': None,
          'end_char_idx': 7604,
          'excluded_embed_metadata_keys': [],
          'excluded_llm_metadata_keys': [],
          'id_': 'fb307225-792c-4e50-9f0d-d4b583958577',
          'metadata': {'airline': 'American Airlines',
                       'policy': 'Restricted Items Policy',
                       'topic': 'What can you fly with?'},
          'metadata_seperator': '\n',
          'metadata_template': '{key}: {value}',
          'relationships': {<NodeRelationship.SOURCE: '1'>: {'class_name': 'RelatedNodeInfo',
                                                             'hash': '13f527756df7c4084995d5974c74f8508e5679a0d89a650b3cd08461cc6aa96c',
                                                             'metadata': {'airline': 'American '
                                                                                     'Airlines',
                                

In [59]:
question = "What is the baggage fee for economy class for American Airline?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))

nodes = kg_index.as_retriever().retrieve(question)
for node in nodes:
    print("NODE:")
    pprint(node.dict())
    if hasattr(node, "relationships"):
        for key, rel in node.relationships.items():
            print("RELATION: ", key)
            pprint(rel.dict())


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the baggage fee for economy class for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._ba

<b>Q: What is the baggage fee for economy class for American Airline?<b>

<b>A: The baggage fee for economy class on American Airlines varies depending on the specific fare type and route. It is recommended to check the airline's official website or contact their customer service for the most up-to-date and accurate information regarding baggage fees for economy class.</b>

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the baggage fee for economy class for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:recei

In [60]:
question = "What is the bag fee for checked bag for American Airline?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))

nodes = kg_index.as_retriever().retrieve(question)
for node in nodes:
    print("NODE:")
    pprint(node.dict())
    if hasattr(node, "relationships"):
        for key, rel in node.relationships.items():
            print("RELATION: ", key)
            pprint(rel.dict())


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the bag fee for checked bag for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends

<b>Q: What is the bag fee for checked bag for American Airline?<b>

<b>A: The bag fee for a checked bag on American Airlines is not explicitly mentioned in the provided context information.</b>

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the bag fee for checked bag for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_res

In [65]:
nl2kg_query_engine = KnowledgeGraphQueryEngine(
    graph_query_synthesis_prompt=DEFAULT_NEO4J_NL2CYPHER_PROMPT,
    llm=llm,
    refresh_schema=True,
    storage_context=storage_context,
    verbose=True,
    graph_store_query_depth=3,
)

kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="hybrid",
    verbose=True,
    response_mode="tree_summarize",
    graph_store_query_depth=4,
)

question = "What is the bag fee for checked bag for American Airline?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))

if False:
    nodes = kg_index.as_retriever().retrieve(question)
    for node in nodes:
        print("NODE:")
        pprint(node.dict())
        if hasattr(node, "relationships"):
            for key, rel in node.relationships.items():
                print("RELATION: ", key)
                pprint(rel.dict())


DEBUG:llama_index.graph_stores.nebula.base:get_schema()
schema: Node properties: [{'tag': 'entity', 'properties': [('name', 'string')]}]
Edge properties: [{'edge': 'relationship', 'properties': [('name', 'string')]}]
Relationships: ['(:entity)-[:relationship]->(:entity)']

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the bag fee for checked bag for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.

<b>Q: What is the bag fee for checked bag for American Airline?<b>

<b>A: The bag fee for a checked bag on American Airlines is free for certain fare types, such as basic economy.</b>

In [68]:

kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="hybrid",
    verbose=True,
    response_mode="tree_summarize",
    graph_store_query_depth=4,
)

question = "What is the bag fee for checked bag for American Airline?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the bag fee for checked bag for American Airline?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends

<b>Q: What is the bag fee for checked bag for American Airline?<b>

<b>A: The bag fee for a checked bag on American Airlines is to pay online within 24 hours of departure.</b>

In [69]:

question = "What is the fee for checked bag for American Airlines?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))

if False:
    nodes = kg_index.as_retriever().retrieve(question)
    for node in nodes:
        print("NODE:")
        pprint(node.dict())
        if hasattr(node, "relationships"):
            for key, rel in node.relationships.items():
                print("RELATION: ", key)
                pprint(rel.dict())


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the fee for checked bag for American Airlines?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2b8a61190>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x2b2bd3410> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2b86a5dd0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 08 May 2024 16:38:20 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-hrq

<b>Q: What is the fee for checked bag for American Airlines?<b>

<b>A: The fee for a checked bag on American Airlines varies depending on the specific route and fare type. It is recommended to check the most up-to-date information on American Airlines' official website or contact their customer service for the latest details on checked bag fees.</b>

In [70]:
question = "What is the fee for checked bag for American Airlines?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the fee for checked bag for American Airlines?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sy

<b>Q: What is the fee for checked bag for American Airlines?<b>

<b>A: The fee for a checked bag on American Airlines varies depending on the specific fare type and route. It is recommended to check the most up-to-date information on American Airlines' official website or contact their customer service for the latest details on checked bag fees.</b>

In [71]:
question = "What is the fee for checked bag for economy class to Cuba on American Airlines?"
query_response = kg_index_query_engine.query(question)

display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {query_response}</b>"))


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nWhat is the fee for checked bag for economy class to Cuba on American Airlines?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_val

<b>Q: What is the fee for checked bag for economy class to Cuba on American Airlines?<b>

<b>A: The fee for a checked bag for economy class to Cuba on American Airlines is $150 for the 2nd bag, except during the seasonal period from November 16 to January 9 when the fee is $200.</b>

In [73]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="user1",
)

chat_engine = kg_index.as_chat_engine(memory=chat_memory)

In [74]:
response = chat_engine.chat("What is the fee for a checked bag on American Airlines?")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is the fee for a checked bag on American Airlines?'}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'query_engine_tool', 'description': 'Useful for running a natural language query\nagainst a knowledge base and get back a natural language response.\n', 'parameters': {'type': 'object', 'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input']}}}]}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_valu

In [75]:
display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response}</b>"))


<b>Q: What is the fee for checked bag for economy class to Cuba on American Airlines?<b>

<b>A: The fee for a checked bag on American Airlines varies depending on the destination. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online). For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35.</b>

In [76]:
question_2 = "I'm flying economy class to Cuba."
response_2 = chat_engine.chat(question_2)
display(Markdown(f"<b>Q: {question_2}<b>\n\n<b>A: {response_2}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is the fee for a checked bag on American Airlines?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_Gd1fV21PI20IzDWbsRnRo0f3', 'function': {'arguments': '{"input":"What is the fee for a checked bag on American Airlines?"}', 'name': 'query_engine_tool'}, 'type': 'function'}]}, {'role': 'tool', 'content': 'The fee for a checked bag on American Airlines varies depending on the destination and when the ticket was issued. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online). For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35.', 'name': 'query_engine_tool', 'tool_call_id': 'call_Gd1fV21PI20IzDWbsRnRo0f3'}, {'role': 'assistant', 'content': 'The fee for a checked bag on American

<b>Q: I'm flying economy class to Cuba.<b>

<b>A: The fee for a checked bag on American Airlines for economy class to Cuba is $150 for the 2nd bag, except for Main Plus tickets issued on/after September 20, 2023, which have an extra free checked bag.</b>

In [77]:
display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response}</b>"))
display(Markdown(f"<b>Q: {question_2}<b>\n\n<b>A: {response_2}</b>"))

<b>Q: What is the fee for checked bag for economy class to Cuba on American Airlines?<b>

<b>A: The fee for a checked bag on American Airlines varies depending on the destination. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online). For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35.</b>

<b>Q: I'm flying economy class to Cuba.<b>

<b>A: The fee for a checked bag on American Airlines for economy class to Cuba is $150 for the 2nd bag, except for Main Plus tickets issued on/after September 20, 2023, which have an extra free checked bag.</b>